<a href="https://colab.research.google.com/github/sherxon/kaggle/blob/main/digit_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
import pandas as pd 
import numpy as np 
from keras.layers import Dense, Input, Conv2D, BatchNormalization, Activation,MaxPooling2D, AveragePooling2D, Flatten
from keras.models import Model
from keras.utils import to_categorical

In [11]:
train_df = pd.read_csv('/content/drive/My Drive/kaggle/digit_analyzer/train.csv')
test_df = pd.read_csv('/content/drive/My Drive/kaggle/digit_analyzer/test.csv')

In [44]:
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [45]:
train_np = train_df.to_numpy().astype('int32')
test_x = test_df.to_numpy().astype('int32')

In [46]:
train_x = train_np[:, 1:]
train_y=train_np[:,0]

In [47]:
train_x=np.reshape(train_x, (42000, -1, 28))/255.0
test_x= np.reshape(test_x, (test_x.shape[0], -1, 28))/255.0

In [49]:
train_x=np.reshape(train_x, list(train_x.shape)+[1])
test_x=np.reshape(test_x, list(test_x.shape)+[1])

In [84]:
inputs= Input(shape=(28,28,1))

X=Conv2D(filters=28, kernel_size=(3,3), input_shape=(28,28,1))(inputs)
X=BatchNormalization(axis=3)(X)
X=Activation('relu')(X)
X=MaxPooling2D(pool_size=(3,3), strides=(2,2))(X)

X=Conv2D(56, (3,3))(X)
X=BatchNormalization(axis=3)(X)
X=Activation('relu')(X)

X=Conv2D(112, (3,3))(X)
X=BatchNormalization(axis=3)(X)
X=Activation('relu')(X)

X=Conv2D(224, (3,3))(X)
X=BatchNormalization(axis=3)(X)
X=Activation('relu')(X)

X=AveragePooling2D()(X)
X=Flatten()(X)
X=Dense(20, activation='relu')(X)
X=Dense(10, activation='softmax')(X)

model = Model(inputs, X)


In [85]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [86]:
model.summary()

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 26, 26, 28)        280       
_________________________________________________________________
batch_normalization_18 (Batc (None, 26, 26, 28)        112       
_________________________________________________________________
activation_18 (Activation)   (None, 26, 26, 28)        0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 12, 12, 28)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 10, 10, 56)        14168     
_________________________________________________________________
batch_normalization_19 (Batc (None, 10, 10, 56)      

In [92]:
train_y=to_categorical(train_y, 10)
model.fit(train_x, train_y, batch_size=32, epochs=10)

Epoch 1/10
1313/1313 [==============================] - 139s 106ms/step - loss: 0.1141 - accuracy: 0.9651
Epoch 2/10
1313/1313 [==============================] - 138s 105ms/step - loss: 0.0500 - accuracy: 0.9853
Epoch 3/10
1313/1313 [==============================] - 142s 108ms/step - loss: 0.0379 - accuracy: 0.9885
Epoch 4/10
1313/1313 [==============================] - 141s 107ms/step - loss: 0.0317 - accuracy: 0.9903
Epoch 5/10
1313/1313 [==============================] - 138s 105ms/step - loss: 0.0245 - accuracy: 0.9925
Epoch 6/10
1313/1313 [==============================] - 139s 106ms/step - loss: 0.0219 - accuracy: 0.9932
Epoch 7/10
1313/1313 [==============================] - 140s 106ms/step - loss: 0.0198 - accuracy: 0.9940
Epoch 8/10
1313/1313 [==============================] - 141s 108ms/step - loss: 0.0168 - accuracy: 0.9946
Epoch 9/10
1313/1313 [==============================] - 139s 106ms/step - loss: 0.0159 - accuracy: 0.9950
Epoch 10/10
1313/1313 [=======================

In [93]:
result = model.predict(test_x, batch_size=64)

In [94]:
result.shape

(28000, 10)

In [95]:
result2=np.argmax(result, axis=1)
result2.shape

(28000,)

In [96]:
result2[:10]

array([2, 0, 9, 0, 3, 7, 0, 3, 0, 3])

In [97]:
output_df= pd.DataFrame(result2);
output_df.dtypes

0    int64
dtype: object

In [100]:
output_df.columns=['Label']

In [105]:
output_df['ImageId']=range(1, len(result2)+1)
output_df = output_df.reset_index(drop=True)

In [106]:
output_df.head()

,Label,ImageId
0,2,1
1,0,2
2,9,3
3,0,4
4,3,5


In [108]:
output_df.to_csv('/content/drive/My Drive/kaggle/digit_analyzer/output_1.csv', index=False)